## ARIMA (AutoRegressive Integrated Moving Average)

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

def generate_anomalous_data(num_clients=30, num_months=45, anomaly_rate=0.1, anomaly_magnitude=1.5):
    months = pd.date_range(start='2018-01', periods=num_months, freq='M')
    client_data = {}

    for i in range(num_clients):
        base_revenue = np.random.uniform(800 + i * 500, 1500 + i * 500, num_months)
        anomalies = np.zeros(num_months)
        num_anomalies = int(anomaly_rate * num_months)

        if num_anomalies > 0:
            anomaly_indices = np.random.choice(num_months, size=num_anomalies, replace=False)
            anomalies[anomaly_indices] = base_revenue[anomaly_indices] * (anomaly_magnitude - 1)

        revenue = base_revenue + anomalies
        client_data[f"client_{i+1}"] = pd.DataFrame({'Month': months, 'Revenue': revenue})

    return client_data

def detect_arima_anomalies(original_data, predictions):
    residual = original_data['Revenue'].sub(predictions, fill_value=0)
    std_residual = residual.std()
    anomalies = original_data.loc[np.abs(residual) > 2 * std_residual]
    anomalies['Residual'] = residual.loc[anomalies.index]
    return anomalies

# Generate the data
client_data = generate_anomalous_data()

# Analyze the data
anomaly_results = {}

for client in client_data:
    # Build and fit the ARIMA model
    model = ARIMA(client_data[client]['Revenue'], order=(1, 1, 1))
    result = model.fit()

    # Make predictions
    predictions = result.predict(start=1, end=len(client_data[client]['Revenue']))
    
    # Detect anomalies
    anomalies = detect_arima_anomalies(client_data[client], predictions)

    # Store the results
    anomaly_results[client] = anomalies

for client, anomalies in anomaly_results.items():
    print(f"{client}: {len(anomalies)} anomalies")
    print(anomalies)


/Users/Dhaval/opt/miniconda3/envs/ml/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/Dhaval/opt/miniconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


client_1: 1 anomalies
       Month      Revenue     Residual
0 2018-01-31  1426.173873  1426.173873
client_2: 1 anomalies
       Month     Revenue    Residual
0 2018-01-31  1889.68909  1889.68909
client_3: 2 anomalies
        Month      Revenue     Residual
0  2018-01-31  2256.885682  2256.885682
44 2021-09-30  3624.809378  1400.115649
client_4: 1 anomalies
       Month      Revenue     Residual
0 2018-01-31  3716.172735  3716.172735
client_5: 2 anomalies
        Month      Revenue     Residual
0  2018-01-31  3330.139347  3330.139347
11 2018-12-31  4915.425982  1779.230811
client_6: 1 anomalies
       Month      Revenue     Residual
0 2018-01-31  3977.499919  3977.499919
client_7: 2 anomalies
        Month      Revenue     Residual
0  2018-01-31  4251.163521  4251.163521
29 2020-06-30  6727.115584  2519.123119
client_8: 2 anomalies
       Month      Revenue     Residual
0 2018-01-31  4799.244221  4799.244221
5 2018-06-30  7487.868718  3049.903513
client_9: 2 anomalies
        Month    